In [1]:
import torch

print(torch.__version__)

2.1.2+cu121


In [1]:
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

from cust_dataset import CustDataset
from network import Network
from torch.utils import *

from sklearn.model_selection import ShuffleSplit
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import roc_auc_score
import argparse
import numpy as np
import pandas as pd

In [2]:
data = CustDataset()

Initializing CustomDataset 1
Number of rows selected: 4000
Loaded labels...
sss done
CustomDataset initialized.


In [9]:
max = np.max(data.vars['new_score'])
min = np.min(data.vars['new_score'])
print(min, max)

-4.117182928739916 3.663844091250903


In [2]:
# torch.set_default_dtype(torch.float32)
# parser = argparse.ArgumentParser()
# parser.add_argument('job_id',type=str)
# args = parser.parse_args()
# print(args.job_id)
# print('number of gpus ',torch.cuda.device_count())

In [3]:
# directory = args.job_id
# parent_directory = '/data/users2/pnadigapusuresh1/JobOutputs'
# path = os.path.join(parent_directory,directory)
# model_save_path = os.path.join(path,'models_fold')

# if not os.path.exists(path):
#     os.mkdir(path)
#     os.mkdir(model_save_path)

In [4]:
torch.manual_seed(52)
# number of subprocesses to use for data loading
num_workers = 4
# how many samples per batch to load
batch_size = 15
# if torch.cuda.device_count() > 1:
#     batch_size *= torch.cuda.device_count()
# else:
#     batch_size = 15
# percentage of training set to use as validation
valid_size = 0.20
# percentage of data to be used for testset
test_size = 0.10

In [5]:
batch_size = 5

In [6]:

test_data = CustDataset(train=False,valid=False)

Initializing CustomDataset 1
Number of rows selected: 500
data src_subject_id
NDARINV005V6D2C   -2.640752
NDARINV007W6H7B    1.337910
NDARINV00CY2MDM    1.058453
NDARINV00HEV6HB    0.508555
NDARINV00LJVZK2    0.509983
                     ...   
NDARINV1VMB1ZYW    3.450872
NDARINV1VMJZV9V   -0.359201
NDARINV1VRGHTD2   -1.741937
NDARINV1VXVRPHJ    0.512647
NDARINV1W2D4CFP   -0.189944
Name: new_score, Length: 500, dtype: float64
Loaded labels...
sss done
CustomDataset initialized.


In [8]:
test_data

In [8]:
# get filtered variables
vars = test_data.vars

In [9]:
# Prepare for k-fold

sss = ShuffleSplit(n_splits=5,test_size=0.2,random_state=52)
learning_rate = 0.00001
fold = 1

In [10]:
next(sss.split(test_data.train_idx))

(array([ 27, 103,  88,  83,  55,   4,  44,  62,  93,  60,  14,  50,  66,
         29,  95,  41,  58, 114,  77,  37,  96, 112, 111,  82,   3, 113,
         57,  89, 101,  67, 105, 100,  84,  12,  56,  81,  98,  53,   1,
        116,  10,  80,  17,   9,  49,  30,  38,   0,  26,  15,  25,   6,
         65,  85,  59,  64,  74,  78,  43,  34,  20, 108,   7,  90,  71,
         22,  39,  63,  76, 104,  79,  45,  61,  42,  46,  54, 109,  16,
          5,  33, 110,  97,  99,  35,  91, 119, 106,  69,  32, 118,  86,
         23,  13,  11,  28, 117]),
 array([107,  52, 102,  18,  94,  47,  36, 115,  92,  68,  70,   2,  19,
         24,  72,  21,  40,  75,  73,  87,   8,  31,  48,  51]))

In [11]:
train_sampler = SubsetRandomSampler(test_data.train_idx)
valid_sampler = SubsetRandomSampler(test_data.test_idx)

In [12]:
train_loader = DataLoader(test_data,batch_size=batch_size, 
                                sampler= train_sampler, num_workers=num_workers)
valid_loader = DataLoader(test_data,batch_size=batch_size, 
                                sampler= valid_sampler, num_workers=num_workers)

In [13]:
model = Network()

In [14]:
X,y = next(iter(train_loader))

In [18]:
out = model(torch.unsqueeze(X, 1).float())

In [19]:
out

tensor([[-0.2712],
        [-0.2299],
        [-0.2923],
        [-0.2592],
        [-0.2562]], grad_fn=<AddmmBackward0>)

In [9]:
for train_idx, valid_idx in sss.split(train_data.train_idx):
    # writer = SummaryWriter(log_dir=path+'/fold'+str(fold))

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_data.test_idx)

    train_loader = DataLoader(train_data,batch_size=batch_size, 
                                sampler= train_sampler, num_workers=num_workers)
    valid_loader = DataLoader(valid_data,batch_size=batch_size, 
                                sampler= valid_sampler, num_workers=num_workers)

    test_loader = DataLoader(test_data,batch_size=batch_size,
                                sampler= test_sampler, num_workers=num_workers)

    device = (
        "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    print(f"Using {device} device")

    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # print(f"Using {device} device")         

    model = Network()
   
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    print(model)

    #%%

    epochs = 10
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    #%%

    model.to(device)

    print('Starting to Train...')

    for e in range(1,epochs+1):
        size = len(train_loader)
        print("train size", size)
        model.train()
        train_loss = 0
        # with torch.no_grad():
        for X, y in train_loader:
            X, y = X.to(device).float(), y.to(device).float()
            X = torch.unsqueeze(X, 1).float() 
            pred = model(X)
            pred = pred.squeeze()  
            loss = criterion(pred, y)
            train_loss += ((loss.item())*X.shape[0])

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        avg_train_loss = train_loss / len(train_idx)
        # avg_train_loss = train_loss / len(train_loader)
        # print(f"Average train loss: {avg_train_loss}")

        #<!------Valid------->
        # else:
        size = len(valid_loader)
        print("Valid size", size)
        model.eval()
        valid_loss = 0

        with torch.no_grad():
            for X,y in valid_loader:
                X, y = X.to(device).float(), y.to(device).float()
                X = torch.unsqueeze(X, 1).float() 
                pred = model(X)
                pred = pred.squeeze()  
                loss = criterion(pred, y)
                valid_loss += ((loss.item())*X.shape[0])

        avg_valid_loss = valid_loss / len(valid_idx)
        # avg_valid_loss = valid_loss / len(valid_loader)
        # print(f"Average valid loss: {avg_valid_loss}")


        #<!------Test-------->
        # size = len(test_loader)
        # print("test size", size)
        # model.eval()
        # test_loss = 0

        # with torch.no_grad():
        #     for X, y in test_loader:
        #         X, y = X.to(device).float(), y.to(device).float()
        #         X = torch.unsqueeze(X, 1).float() 
        #         pred = model(X)
        #         pred = pred.squeeze()  
        #         test_loss += criterion(pred, y).item()

        # avg_test_loss = test_loss / len(test_loader)
        # print(f"Average test loss: {avg_test_loss}")

        
        print("Epoch: {}/{}.. ".format(e, epochs))

        print('Train Loss', train_loss/len(train_loader),e)
        print('Validation Loss', valid_loss/len(valid_loader),e)
        # print('Test Loss', test_loss/len(test_loader),e)
        

    # fold+=1
    print('####################################################################')
    print("Done")

Using cpu device
Network(
  (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=1, bias=True)
  (d3d): Dropout3d(p=0.2, inplace=False)
  (layer2)

Valid size 1
Epoch: 1/10.. 
Train Loss 3.2995426654815674 1
Validation Loss 1.3055212497711182 1
train size 1
Valid size 1
Epoch: 2/10.. 
Train Loss 166.49395751953125 2
Validation Loss 0.6933380365371704 2
train size 1
Valid size 1
Epoch: 3/10.. 
Train Loss 5.317201137542725 3
Validation Loss 0.4049515426158905 3
train size 1
Valid size 1
Epoch: 4/10.. 
Train Loss 44.45218276977539 4
Validation Loss 0.3655177056789398 4
train size 1
Valid size 1
Epoch: 5/10.. 
Train Loss 39.626830101013184 5
Validation Loss 0.3829321265220642 5
train size 1
Valid size 1
Epoch: 6/10.. 
Train Loss 20.568161487579346 6
Validation Loss 0.4155452251434326 6
train size 1
Valid size 1
Epoch: 7/10.. 
Train Loss 1.5559585690498352 7
Validation Loss 0.41929036378860474 7
train size 1
Valid size 1
Epoch: 8/10.. 
Train Loss 9.55005955696106 8
Validation Loss 0.3845386207103729 8
train size 1
Valid size 1
Epoch: 9/10.. 
Train Loss 6.065716981887817 9
Validation Loss 0.33692383766174316 9
train size 1
Valid size 1


KeyboardInterrupt: 